In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import gradio as gr

# Load data
df = pd.read_csv("data.csv")
df['Date'] = pd.to_datetime(df['Date'])

# Forecast function
def forecast_sales(category, steps=30):
    steps = int(steps)
    df_cat = df[df['Product_Category'] == category]
    daily_sales = df_cat.groupby('Date')['Total_Amount'].sum()
    daily_sales.index = pd.DatetimeIndex(daily_sales.index, freq='D')

    # Train/test split
    train = daily_sales[:-steps]
    test = daily_sales[-steps:]

    # Fit SARIMA
    model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,7))
    fit_model = model.fit(disp=False)

    # Forecast
    pred = fit_model.get_forecast(steps=steps)
    pred_values = pred.predicted_mean

    # Accuracy metrics
    mae = mean_absolute_error(test, pred_values)
    rmse = np.sqrt(mean_squared_error(test, pred_values))
    accuracy_pct = (1 - mae / np.mean(test)) * 100

    # Trend insight
    trend = np.mean(pred_values) - np.mean(train[-7:])
    if trend > 0:
        trend_text = "upward trend. Stock may need to be increased."
    elif trend < 0:
        trend_text = "downward trend. Consider promotions to avoid overstock."
    else:
        trend_text = "stable. Maintain current stock levels."

    # Prepare output
    forecast_list = [f"{date.date()}: {round(value,2)}" for date, value in zip(test.index, pred_values)]
    output_text = f"Forecast for {category} ({steps} days):\n" + \
                  "\n".join(forecast_list) + \
                  f"\n\nTrend Insight: {trend_text}" + \
                  f"\nMAE: {mae:.2f}, RMSE: {rmse:.2f}, Accuracy: {accuracy_pct:.2f}%"

    return output_text

# Gradio interface
demo = gr.Interface(
    fn=forecast_sales,
    inputs=[
        gr.Dropdown(label="Product Category", choices=df['Product_Category'].unique().tolist(), value='Electronics'),
        gr.Number(label="Forecast Days", value=30)
    ],
    outputs=gr.Textbox(label="Predicted Sales"),
    title="📈 Sales Forecast Dashboard",
    description="Select a product category and forecast horizon to see daily predicted sales, trend insight, and accuracy metrics.",
    theme="soft"
)

demo.launch(share=True)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
